In [1]:
import json, random

# Define the example data
example_data = {
    'Filtering': ['Low Abundance Filtering', 'Prevalence Filtering', 'Variance Filtering', 'No Filtering'],
    'Zero-Handling': ['Pseudocount Addition', 'k-NN Imputation', 'No Zero-Handling'],
    'Normalization': ['TSS', 'CSS', 'TMM', 'CLR', 'No Normalization'],
    'Transformation': ['Log', 'Logit', 'AST', 'No Transformation'],
    'Model Perturbation': ['deseq2', 'edger', 'maaslin2', 'aldex2', 'metagenomeseq']
}

# Define the cross-step mutually exclusive options
crossStepMutuallyExclusiveOptions = {
    'Normalization': {
        'CLR': { 'Transformation': ['Log', 'Logit'], 'Zero-Handling': ['No Zero-Handling'] },
        'TMM': { 'Transformation': ['Logit'] },
        'CSS': { 'Transformation': ['Logit'], 'Model Perturbation': ['metagenomeseq'] },
        'No Normalization': { 'Transformation': ['AST'] }
    },
    'Transformation': {
        'Log': { 'Normalization': ['CLR'], 'Zero-Handling': ['No Zero-Handling'] },
        'Logit': { 'Normalization': ['CLR', 'TMM', 'CSS'], 'Zero-Handling': ['No Zero-Handling'] },
        'AST': { 'Normalization': ['No Normalization'] }
    },
    'Zero-Handling': {
        'No Zero-Handling': { 'Transformation': ['Log', 'Logit'], 'Normalization': ['CLR'] }
    },
    'Model Perturbation': {
        'metagenomeseq': { 'Normalization': ['CSS'] }
    }
}

# List of steps in hierarchical order
steps = [
    'Filtering',
    'Zero-Handling',
    'Normalization',
    'Transformation',
    'Model Perturbation'
]

# Initialize the root of the JSON structure
tree = {
    "name": "Raw data",
    "children": []
}

# Function to recursively build the tree
def build_tree(current_node, steps, data, leaf_ids, current_step=0, path=None):
    if current_step >= len(steps):
        return
    if path is None:
        path = {}
    step = steps[current_step]
    for option in data.get(step, []):
        # Check if the current option is compatible with the existing path
        if is_compatible(path, step, option):
            node = {"name": option}
            new_path = path.copy()
            new_path[step] = option
            if current_step == len(steps) - 1:
                # Assign a unique ID to each leaf node
                node["id"] = f"leaf-{leaf_ids[0]}"
                leaf_ids[0] += 1
            else:
                node["children"] = []
                build_tree(node, steps, data, leaf_ids, current_step + 1, new_path)
            current_node["children"].append(node)

def is_compatible(path, current_step, current_option):
    for step, option in path.items():
        if step in crossStepMutuallyExclusiveOptions and option in crossStepMutuallyExclusiveOptions[step]:
            exclusions = crossStepMutuallyExclusiveOptions[step][option]
            if current_step in exclusions and current_option in exclusions[current_step]:
                return False
    return True

def extract_leaf_ids(node, id_dict):
    
    if 'children' not in node or not node['children']:
        if 'id' in node:
            id_dict[node['id']] = {
                "data_point": [round(random.uniform(0, 10), 2), round(random.uniform(0, 10), 2)]
            }
    else:
        for child in node['children']:
            extract_leaf_ids(child, id_dict)

# Initialize leaf ID counter
leaf_ids = [1]

# Build the hierarchical tree
build_tree(tree, steps, example_data, leaf_ids)

# Save the tree to a JSON file
with open('src/renderer/src/public/data.json', 'w') as f:
    json.dump(tree, f, indent=2)

with open('src/renderer/src/public/data.json', 'r') as f:
    data = json.load(f)

leaf_id_data = {}
extract_leaf_ids(data, leaf_id_data)

with open('src/renderer/src/public/leaf_id_data_points.json', 'w') as f:
    json.dump(leaf_id_data, f, indent=2)

print("Success.")

Success.
